In [1]:
import numpy as np
import text_normalizer as tn
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
file_info = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_data_v1.csv"
df = pd.read_csv(file_info, sep='\t')

In [3]:
df.head()

,url,code,category,language,title,summary,key_words,content,combined,clean_combined
0,*://BLAZINHERB.CA,200,dr,en,Blazin Herb,The Diamond High Crew,Blazin herb,Blazin Herb THC Edibles Concentrates Flower/bu...,Blazin Herb The Diamond High Crew Blazin herb ...,blazin herb diamond high crew blazin herb blaz...
1,*://THEHONEYPOTSTORE.CA,200,dr,en,The Honey Pot Cannabis and Medical Marijuana p...,"The Honey Pot Hamilton Ontario, provides canna...","The Honey Pot ,Georgia Peach Cannabis and Medi...",The Honey Pot Home About Us F.A.Q.'s About The...,The Honey Pot Cannabis and Medical Marijuana p...,honey pot cannabis medical marijuana product s...
2,*://AMANTE.CA,200,dr,en,Amante Organics - Licensed Certified Organic C...,np-template-header-footer-from-plugin,"Organic Cannabis, Certified Organic, Weed, Org...",Amante Organics LOCAL | ORGANIC | SUSTAINABLE ...,Amante Organics - Licensed Certified Organic C...,amante organics licensed certified organic can...
3,*://CANNAFLY.CA,200,dr,en,Cannafly,Cannafly,"Marijuana, cannabis, medical marijuana, dispen...",Vancouver's Premium Mobile Medicinal Cannabis ...,"Cannafly Cannafly Marijuana, cannabis, medical...",cannafly cannafly marijuana cannabis medical m...
4,*://GRON.VC,200,dr,en,Grøn Ventures,We are backing businesses and brands with pote...,"cannabis vc, cannabis venture capital, gron ve...","GRØN VENTURES 4675 MacArthur Ct, 15th Floor, N...",Grøn Ventures We are backing businesses and br...,grn ventures back business brand potential mar...


In [4]:
df.groupby(df.category).size()

category
bu    11731
dr     1595
ed     3894
mk     8775
os     6789
sp     2535
sx     6557
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
#get 10% holdout set
train_corpus, holdout_corpus, train_label, holdout_label = train_test_split(np.array(df['clean_combined']), 
                                                                            np.array(df['category']),
                                                                           test_size=.10,random_state=1234)

#get 80% train and 20% test sets
train_corpus, test_corpus, train_label, test_label = train_test_split(train_corpus,
                                                                      train_label,
                                                                      test_size=.2225,random_state=1234) #.9 * .2225 = .20

train_corpus.shape, test_corpus.shape, holdout_corpus.shape

((29302,), (8386,), (4188,))

In [6]:
from collections import Counter
trd = dict(Counter(train_label))
tsd = dict(Counter(test_label))
tvd = dict(Counter(holdout_label))

(pd.DataFrame([[key,trd[key],tsd[key],tvd[key]] for key in trd],columns=['Target Label','Train Count','Test Count','Holdout Count']).sort_values(by=['Train Count','Test Count','Holdout Count'],ascending=False))

,Target Label,Train Count,Test Count,Holdout Count
2,bu,8187,2355,1189
0,mk,6177,1709,889
1,os,4743,1393,653
5,sx,4544,1345,668
6,ed,2723,783,388
4,sp,1783,494,258
3,dr,1145,307,143


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

#build BOW features on train corpus
tv = TfidfVectorizer(use_idf=True, min_df=0.0,max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)
#transform test corpus into features
tv_test_features = tv.transform(test_corpus)
#transform holdout corpus into features
tv_holdout_features = tv.transform(holdout_corpus)

print('TFIDF model:> Train features shape:',tv_train_features.shape,'Test features shape:',tv_test_features.shape,'Holdout features shape:',tv_holdout_features.shape)

TFIDF model:> Train features shape: (29302, 364296) Test features shape: (8386, 364296) Holdout features shape: (4188, 364296)


In [10]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(n_estimators=10,random_state=1234)
rfc.fit(tv_train_features,train_label)


RandomForestClassifier(n_estimators=10, random_state=1234)

In [11]:
rfc_tfidf_cv_scores = cross_val_score(rfc,tv_train_features,train_label,cv=5)

In [12]:
rfc_tfidf_cv_mean_score = np.mean(rfc_tfidf_cv_scores)

In [13]:
print('CV Accuracy (5-fold):',rfc_tfidf_cv_scores)
print('Mean CV Accuracy:',rfc_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.72769152 0.7177956  0.72372014 0.73668942 0.73412969]
Mean CV Accuracy: 0.7280052734771931


In [14]:
rfc_tfidf_test_score = rfc.score(tv_test_features,test_label)
print('Test Accuracy:',rfc_tfidf_test_score)

Test Accuracy: 0.7356308132601955


In [15]:
#SVM
from sklearn.svm import LinearSVC

svm = LinearSVC(penalty='l2',C=1,random_state=1234)
svm.fit(tv_train_features,train_label)

LinearSVC(C=1, random_state=1234)

In [16]:
svm_tfidf_cv_scores = cross_val_score(svm,tv_train_features,train_label,cv=5)

In [17]:
svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)

In [18]:
print('CV Accuracy (5-fold):',svm_tfidf_cv_scores)
print('Mean CV Accuracy:',svm_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.84268896 0.84576011 0.85153584 0.84931741 0.84539249]
Mean CV Accuracy: 0.8469389607825896


In [19]:
svm_tfidf_test_score = svm.score(tv_test_features,test_label)
print('Test Accuracy:',svm_tfidf_test_score)

Test Accuracy: 0.8578583353207727


In [20]:
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=1)
mnb.fit(tv_train_features,train_label)


MultinomialNB(alpha=1)

In [21]:
mnb_tfidf_cv_scores = cross_val_score(mnb,tv_train_features,train_label,cv=5)

In [22]:
mnb_tfidf_cv_mean_score = np.mean(mnb_tfidf_cv_scores)

In [23]:
print('CV Accuracy (5-fold):',mnb_tfidf_cv_scores)
print('Mean CV Accuracy:',mnb_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.66353865 0.66985156 0.66143345 0.66962457 0.67116041]
Mean CV Accuracy: 0.6671217272967083


In [25]:
nmb_tfidf_test_score = mnb.score(tv_test_features,test_label)
print('Test Accuracy:',nmb_tfidf_test_score)

Test Accuracy: 0.6819699499165276


In [26]:
import model_evaluation_utils as meu
svm_predictions = svm.predict(tv_test_features)
meu.get_metrics(true_labels=test_label,predicted_labels=svm_predictions)

Accuracy: 0.8579
Precision: 0.8584
Recall: 0.8579
F1 Score: 0.858


In [28]:
unique_classes = list(set(test_label))
meu.display_classification_report(true_labels=test_label,predicted_labels=svm_predictions,classes=unique_classes)

              precision    recall  f1-score   support

          bu       0.85      0.86      0.86      2355
          dr       0.99      0.94      0.96       307
          sp       0.90      0.88      0.89       494
          mk       0.75      0.76      0.75      1709
          sx       0.99      0.99      0.99      1345
          os       0.82      0.79      0.80      1393
          ed       0.91      0.90      0.90       783

    accuracy                           0.86      8386
   macro avg       0.89      0.87      0.88      8386
weighted avg       0.86      0.86      0.86      8386



In [29]:
meu.display_confusion_matrix(true_labels=test_label,predicted_labels=svm_predictions,classes=unique_classes)

TypeError: __new__() got an unexpected keyword argument 'labels'